In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

2024-05-22 15:13:24.978838: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
training_data=pd.read_csv('./train.csv',encoding='ISO-8859-1')

In [3]:
training_data

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26
...,...,...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,night,31-45,Ghana,31072940,227540.0,137
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,morning,46-60,Greece,10423054,128900.0,81
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,noon,60-70,Grenada,112523,340.0,331
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,night,70-100,Guatemala,17915568,107160.0,167


In [4]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [5]:
embeddings_dict={}
with open("./glove/glove.twitter.27B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
        except ValueError:
            print(f"Skipping word {word} due to ValueError")

In [7]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [8]:
training_data.dropna(inplace=True)

In [9]:
enc = OneHotEncoder(handle_unknown='ignore')
y_train=enc.fit_transform(training_data['sentiment'].values.reshape(-1,1))

In [10]:
word_to_index={}
i=0
for word in embeddings_dict.keys():
    word_to_index[word]=i
    i+=1

In [11]:
training_data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [12]:
training_data['selected_text'].values

array(['I`d have responded, if I were going', 'Sooo SAD', 'bullying me',
       ..., 'Yay good for both of you.', 'But it was worth it  ****.',
       'All this flirting going on - The ATG smiles. Yay.  ((hugs)'],
      dtype=object)

In [13]:
max_len = max(len(input) for input in training_data['selected_text'])
max_len

158

In [14]:
X=training_data['selected_text'].values

In [15]:
X.shape[0]

27480

In [16]:
 X_indices = np.zeros([X.shape[0],max_len])
    
for i in range(X.shape[0]):                          
        
       
        sentence_words = X[i].lower().split()
        
       
        j = 0
        

        for w in sentence_words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
                j =  j+1

In [17]:
vocab_size = len(word_to_index) + 1    
emb_matrix = np.zeros([vocab_size, 50])

for word, idx in word_to_index.items():
    # Get the word vector
    word_vector = embeddings_dict.get(word)
    
    # Check if the word exists in the embeddings_dict and if the vector has the correct shape
    if word_vector is not None and word_vector.shape[0] == 50:
        emb_matrix[idx, :] = word_vector
    else:
        print(f"Skipping word {word} due to incorrect vector shape")

Skipping word 0.45973 due to incorrect vector shape


In [18]:
embedding_layer = Embedding(vocab_size, 50 ,trainable = False)
embedding_layer.build((None,)) # Do not modify the "None".  This line of code is complete as-is.
# Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
embedding_layer.set_weights([emb_matrix])

In [19]:
X = np.reshape(X, (X.shape[0], 1))
input_shape = X.shape[1:]
sentence_indices = Input(shape=input_shape, dtype='int32')

embeddings = embedding_layer(sentence_indices)   
X = LSTM(units=128, return_sequences=True)(embeddings)
X = Dropout(0.5)(X)
X = LSTM(units=128, return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(3)(X)  # Change the number of units to match the number of classes
X = Activation('softmax')(X)

model = Model(inputs=sentence_indices, outputs=X)
early_stopping = EarlyStopping(monitor='accuracy', patience=2,verbose=1)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
y_train

<27480x3 sparse matrix of type '<class 'numpy.float64'>'
	with 27480 stored elements in Compressed Sparse Row format>

In [ ]:
y_train_dense = y_train.toarray()
history = model.fit(X_indices, y_train_dense, epochs = 100, batch_size = 32, shuffle=True,callbacks=[early_stopping])

Epoch 1/100
859/859 [==============================] - 489s 563ms/step - loss: 1.0925 - accuracy: 0.3964
Epoch 2/100
859/859 [==============================] - 767s 893ms/step - loss: 1.0886 - accuracy: 0.4044
Epoch 3/100
859/859 [==============================] - 790s 919ms/step - loss: 1.0878 - accuracy: 0.4045
Epoch 4/100
859/859 [==============================] - 757s 881ms/step - loss: 1.0876 - accuracy: 0.4045
Epoch 5/100
453/859 [==============>...............] - ETA: 5:50 - loss: 1.0872 - accuracy: 0.4062

In [34]:
test_data=pd.read_csv('./test.csv.xls',encoding='ISO-8859-1')
test_data.dropna(inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3534 entries, 0 to 3533
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            3534 non-null   object 
 1   text              3534 non-null   object 
 2   sentiment         3534 non-null   object 
 3   Time of Tweet     3534 non-null   object 
 4   Age of User       3534 non-null   object 
 5   Country           3534 non-null   object 
 6   Population -2020  3534 non-null   float64
 7   Land Area (Km²)   3534 non-null   float64
 8   Density (P/Km²)   3534 non-null   float64
dtypes: float64(3), object(6)
memory usage: 276.1+ KB


In [39]:
X_test=test_data['text'].values
X_test

array(['Last session of the day  http://twitpic.com/67ezh',
       ' Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).',
       'Recession hit Veronique Branquinho, she has to quit her company, such a shame!',
       ...,
       ' I know what you mean. My little dog is sinking into depression... he wants to move someplace tropical',
       '_sutra what is your next youtube video gonna be about? I love your videos!',
       ' http://twitpic.com/4woj2 - omgssh  ang cute ng bby.!'],
      dtype=object)

In [40]:
X_test_indices = np.zeros([X_test.shape[0],max_len])
    
for i in range(X_test.shape[0]):                              
        
        sentence_words = X_test[i].lower().split()
    
        j = 0

        for w in sentence_words:
            if w in word_to_index:
                X_test_indices[i, j] = word_to_index[w]
                j =  j+1

In [42]:
#test accuracy for higher bias model
y_test=enc.fit_transform(test_data['sentiment'].values.reshape(-1,1))
test_predictions=model.predict(X_test_indices)

111/111 [==============================] - 12s 111ms/step


array([[0.26598513, 0.4235917 , 0.31042317],
       [0.26598513, 0.4235917 , 0.31042314],
       [0.26598513, 0.4235917 , 0.31042317],
       ...,
       [0.26598513, 0.42359173, 0.31042314],
       [0.26598513, 0.42359173, 0.31042314],
       [0.26598513, 0.42359173, 0.31042317]], dtype=float32)

In [50]:
test_predictions

array([[0.26598513, 0.4235917 , 0.31042317],
       [0.26598513, 0.4235917 , 0.31042314],
       [0.26598513, 0.4235917 , 0.31042317],
       ...,
       [0.26598513, 0.42359173, 0.31042314],
       [0.26598513, 0.42359173, 0.31042314],
       [0.26598513, 0.42359173, 0.31042317]], dtype=float32)

In [51]:
y_test

<3534x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3534 stored elements in Compressed Sparse Row format>

In [52]:
test_predictions = np.argmax(test_predictions, axis=1)
y_test = np.argmax(y_test.toarray(), axis=1)

In [54]:
# test set accuracy for higher bias model
accuracy=accuracy_score(y_test,test_predictions)
accuracy

0.4046406338426712

In [24]:
""""Model displays accuracy that is barely better than random guessing (33.3%), definitely underfitted. 
Can decrease bias before trying with textblob"""

'"Model displays accuracy that is barely better than random guessing (33.3%), definitely underfitted. \nCan decrease bias before trying with textblob'

In [66]:
#Lower Bias model 
""""
X = LSTM(units=256, return_sequences=True)(embeddings)
X = Dropout(0.5)(X)
X = LSTM(units=256, return_sequences=True)(X)
X = Dropout(0.5)(X)
X = LSTM(units=256, return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(3)(X)  # Change the number of units to match the number of classes
X = Activation('softmax')(X)

model = Model(inputs=sentence_indices, outputs=X)
early_stopping = EarlyStopping(monitor='accuracy', patience=1, min_delta=0.005,verbose=1)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])"""

In [ ]:
""""history = model.fit(X_indices, y_train_dense, epochs = 100, batch_size = 32, shuffle=True, callbacks=[early_stopping])"""

Epoch 1/100
 34/859 [>.............................] - ETA: 1:01:19 - loss: 1.1245 - accuracy: 0.3483

In [55]:
#Try the sentiment Analysis with TextBlob instead
training_data['sentimentscore'] = training_data['selected_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

In [56]:
def get_sentiment(score):
    if score < 0:
        return 'negative'
    elif score > 0:
        return 'positive'
    else:
        return 'neutral'

training_data['sentiment_guess'] = training_data['sentimentscore'].apply(get_sentiment)

In [57]:
# training data accuracy for TextBlob model
training_accuracy = accuracy_score(training_data['sentiment'], training_data['sentiment_guess'])

In [58]:
training_accuracy

0.5509825327510917

In [60]:
# test data accuracy for TextBlob model
test_data['sentimentscore']=test_data['text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
test_data['sentiment_guess'] = test_data['sentimentscore'].apply(get_sentiment)
test_accuracy=accuracy_score(test_data['sentiment'], test_data['sentiment_guess'])
test_accuracy

0.5919637804187889

In [7]:
""""
#It takes about an hour for the model to come up with the sentiment for all 27480 training examples
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
training_data['roBERTaGuess']=training_data['selected_text'].apply(lambda x: sentiment_analysis(x)[0]['label'])
accuracy = accuracy_score(training_data['sentiment'], training_data['roBERTaGuess'])
accuracy
0.6637554585152838
training_data.to_csv('roBERTa_training',index=False)"""

In [61]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
test_data['roBERTaGuess']=test_data['text'].apply(lambda x: sentiment_analysis(x)[0]['label'])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [65]:
# test set accuracy for roBERTa model
accuracy=accuracy_score(test_data['sentiment'],test_data['roBERTaGuess'])
accuracy

0.7099603848330504